In [ ]:
# using Kernel PyTorch-1.10
# using Kernel PyTorch-1.10
# using Kernel PyTorch-1.10

In [1]:
from gpn.data import GenomeMSA #, Tokenizer
import gpn.model

from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder, StandardScaler
from transformers import AutoModel #, AutoModelForMaskedLM
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import re
import os
import csv
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime, timedelta

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [2]:
model_path = "songlab/gpn-msa-sapiens"
msa_path = "zip:///::89.zarr.zip"
msa_path = "zip:///::/home/sunhuaikuan/ondemand/blue_gpn/examples/msa/89.zarr.zip"
genome_msa = GenomeMSA(msa_path)
model = AutoModel.from_pretrained(model_path).to(device)
model.eval();

Loading MSA...
Loading MSA... Done


Some weights of the model checkpoint at songlab/gpn-msa-sapiens were not used when initializing GPNRoFormerModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias']
- This IS expected if you are initializing GPNRoFormerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPNRoFormerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Main Function to get Embedding

In [3]:
comp = {'A':1, 'C':2, 'G':3, 'T':4}

max_seqlen=128

def Genosome2Embedding(chrom, pos_start, pos_end,  y):  # ref,alt,
    # tokenized msa
    # pos_start = pos - 65
    # pos_end=pos_start + 128
    msa = genome_msa.get_msa(str(chrom), pos_start, pos_end, strand="+", tokenize=True)
    # print(msa.shape)

    msa = torch.tensor(np.expand_dims(msa, 0).astype(np.int64))
    # msa

    # separating human from rest of species
    input_ids, aux_features = msa[:, :, 0], msa[:, :, 1:]
    
    input_ids = input_ids.to(device)
    aux_features = aux_features.to(device)

    
    # with torch.no_grad():
    #     embedding = model(input_ids=input_ids, aux_features=aux_features).last_hidden_state
    # embedding_df = pd.DataFrame(StandardScaler().fit_transform(embedding[0].cpu().numpy()))


    
    # with torch.no_grad():
    #     cls_embedding = model(input_ids=input_ids, aux_features=aux_features).last_hidden_state[:, 0, :]

    #     hidden_states = model(inputs)[0]  # Get model outputs (last hidden states)
    #     cls_embedding = hidden_states.mean(dim=1)  # Shape: (batch_size, embedding_dim)
    # # cls_embedding = outputs.last_hidden_state[:, 0, :]
    # # print(cls_embedding.shape)
    
    # # feature=np.array(embedding_df.iloc[64])
    # feature=np.append(cls_embedding.cpu().numpy(),  [chrom, comp[ref],comp[alt], pos_end-pos_start,  y])



    with torch.no_grad():
        last_hidden_state = model(input_ids=input_ids, aux_features=aux_features).last_hidden_state
        
        # Mean Pooling: Compute the mean across the sequence length (dim=1)
        mean_pooled = last_hidden_state.mean(dim=1)  # Shape: (batch_size, embedding_dim)

    feature=np.append(mean_pooled.cpu().numpy(),  [y]) # chrom,  pos_end-pos_start, comp[ref],comp[alt],
    
    return feature


### Output CSV File

In [4]:
def output2CSV(df, csv_Filename):

    if os.path.exists(csv_Filename):
        os.remove(csv_Filename)

    rows=[]
    # progress_bar = tqdm(total=df.shape[0], desc="Processing")
    for index, row in df.iterrows():
        
        chrom=row['CHROM']
        pos_start=row['START']
        pos_end=row['END']
        # ref=row['REF']
        # alt=row['ALT']        
        # info=row['INFO']
        y=row['y']
        try:
            embedding  =  Genosome2Embedding(chrom, pos_start,pos_end, y) # ref,alt,
            rows.append(embedding)
    
        except Exception as e:
            print(f"exception caught: {e}"+str(row['CHROM'])+'-'+str(row['START']))
    
        # progress_bar.update(1)


        if ((index % 5000) ==0):
            with open(csv_Filename, mode='a', newline='') as file:
                writer = csv.writer(file)
                # writer.writerow(rows)
                for row in rows:
                    writer.writerow(row)
            rows=[]
            # progress_bar.update(1)
            print(f"complete index={index}")


    # df = pd.DataFrame.from_dict(my_dict, orient='index')
    # embed_df.to_csv(csv_Filename, index=False)
    # pd.DataFrame(rows).to_csv(csv_Filename, index=False)
    
    with open(csv_Filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        # writer.writerow(rows)
        for row in rows:
            writer.writerow(row)

    print(f"Create File: "+csv_Filename)
    # progress_bar.reset()
    # progress_bar.close()

In [5]:
### PreProcess Dataset
# def PreProcess(data_filename):
#     # df=pd.read_csv(data_filename, delimiter='\t',  names=['CHROM', 'POS', 'REF','ALT','INFO'])  # No head column row
#     df=pd.read_csv(data_filename, header=0, sep='\t', index_col=None)   # With head column row

#     df=df[(df['REF'].str.len() == 1) & (df['ALT'].str.len() == 1)]
#     df['CHROM'] = df['CHROM'].str.replace('chr', '')
#     df['y']=0
#     return df

# df1=PreProcess('gnomad.v4.1.exon.txt')
# df1
# output2CSV(df1,'./gnomad.v4.1.exon.gpn.csv')

### Load Homo_Sapiens data

In [7]:
max_seqlen=128

# csv_filename='Homo_sapiens.GRCh38.109.txt'
# csv_filename='clinvar_20240805.10missense.txt'
# csv_filename='clinvar_20240805.noncoding.txt'


pathogenecity_type='noncoding'
datafile='/home/sunhuaikuan/ondemand/blue_papers/DNA_LLM_REVIEW/datasets/task04-pathogenecity/clinvar_20240805.'+pathogenecity_type+'.txt'

# pathogenecity_type='missense'
# datafile='/home/sunhuaikuan/ondemand/blue_papers/DNA_LLM_REVIEW/datasets/task04-pathogenecity/clinvar_20240805.'+pathogenecity_type+'_matched.txt'


df=pd.read_csv(datafile, delimiter='\t')

columns_to_keep=['CHROM','POS','Pathogenicity'] # 'ID', 'REF','ALT',
df = df[columns_to_keep]

# Merge CHROM=9 and '9' etc
for i in range(1,23):
    df.loc[df['CHROM']==i,'CHROM']=str(i)

df=df[~df['CHROM'].isna()]
df = df[~df['CHROM'].str.contains('KI')]
df = df[~df['CHROM'].str.contains('GL')]


    

df['START']=df['POS']- max_seqlen //2 -1

df['END']=df['START'] + max_seqlen

df=df[~df['CHROM'].isna()]

Pathogenicity_dict={'B':0,'P':1}
df['y'] = df['Pathogenicity'].map(Pathogenicity_dict)

df=df.drop(columns=['POS','Pathogenicity'])
# df['CHROM'].value_counts()
df

,CHROM,START,END,y
0,1,1020318,1020446,0
1,1,1020325,1020453,0
2,1,1020326,1020454,0
3,1,1020327,1020455,0
4,1,1020327,1020455,0
...,...,...,...,...
95755,X,154990978,154991106,0
95756,X,154992854,154992982,0
95757,X,154993092,154993220,0
95758,X,154997049,154997177,0


In [8]:
%%time

now = datetime.now()
formatted_time = now.strftime("%y-%m-%d-%H-%M-%S")
csv_filename = './pathogenecity_gpn_'+pathogenecity_type+'_'+formatted_time+'.csv'

output2CSV(df,csv_filename)

complete index=0
complete index=5000
complete index=10000
complete index=15000
complete index=20000
complete index=25000
complete index=30000
complete index=35000
complete index=40000
complete index=45000
complete index=50000
complete index=55000
complete index=60000
complete index=65000
complete index=70000
complete index=75000
complete index=80000
complete index=85000
complete index=90000
complete index=95000
Create File: ./pathogenecity_gpn_noncoding_24-11-02-11-22-28.csv
CPU times: user 18min 55s, sys: 3.46 s, total: 18min 59s
Wall time: 20min 13s


### Load CSV File

In [14]:
!pwd

/blue/xiaofan/sunhuaikuan/PAPERS/DNA_LLM_REVIEW/embeddings/pathogenecity


In [16]:
import pandas as pd

csv_filename='/blue/xiaofan/sunhuaikuan/PAPERS/DNA_LLM_REVIEW/embeddings/pathogenecity/embedding-csv/pathogenecity_gpn_noncoding.csv'

def load_embedding_file(csv_filename):

    df=pd.read_csv(csv_filename)
    
    # new_arr = data_array[:, :-4]
    # last_column = data_array[:, -1]
    # data_array = np.column_stack((new_arr, last_column))
    
    column_names = [f'{i}' for i in range(1, df.shape[1])]
    column_names.extend([ 'y'])
    
    df.columns = column_names
    return df

df = load_embedding_file(csv_filename)
df

,1,2,3,4,5,6,7,8,9,10,...,760,761,762,763,764,765,766,767,768,y
0,0.445931,0.805443,0.499554,0.711929,0.321147,-1.092941,-0.199364,0.184887,0.943818,0.292585,...,-0.873815,0.103578,-0.415482,-0.197059,-0.079640,-1.064177,-0.466612,-0.472388,0.848660,0.0
1,0.461076,0.809369,0.483196,0.685488,0.303076,-1.104358,-0.194929,0.190177,0.957248,0.253928,...,-0.877850,0.102479,-0.387591,-0.193366,-0.085549,-1.059689,-0.472710,-0.470236,0.855772,0.0
2,0.513312,0.754226,0.497715,0.698823,0.286727,-1.017692,-0.148586,0.149295,0.969672,0.267036,...,-0.875401,0.040252,-0.378097,-0.163968,-0.135677,-1.037855,-0.503077,-0.470197,0.829473,0.0
3,0.513312,0.754226,0.497715,0.698823,0.286727,-1.017692,-0.148586,0.149295,0.969672,0.267036,...,-0.875401,0.040252,-0.378097,-0.163968,-0.135677,-1.037855,-0.503077,-0.470197,0.829473,0.0
4,0.366928,0.121256,0.285477,1.180893,-0.379903,-0.543935,-0.505797,0.111189,1.207105,0.233208,...,-0.876054,-0.205417,-0.042508,-0.255080,-0.285948,-0.815723,-0.524366,-0.180991,0.821893,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95754,0.960115,-0.405934,-0.963120,0.604153,-1.069464,-0.038014,0.032151,0.178382,-0.173240,-0.667455,...,-1.566620,-0.376819,1.289428,0.359950,-0.283300,-0.062038,-1.202238,0.390646,0.621940,0.0
95755,0.015163,-0.034635,-0.535783,0.233521,-0.565940,-0.448956,-0.256404,0.180318,-0.123829,-0.844805,...,-0.644327,-0.825307,0.556938,-0.078505,-0.648336,-0.629008,-1.000944,0.622980,0.749246,0.0
95756,-0.002326,0.007319,-0.560664,-0.010452,-0.207512,-0.635047,-0.444733,0.240929,-0.179017,-0.806953,...,-0.510489,-0.536669,0.526378,0.160796,-0.679473,-0.535794,-0.854475,0.533914,0.525361,0.0
95757,0.143588,0.178700,-0.840755,0.159178,-0.592489,-0.772373,-0.460265,0.607133,-0.207599,-0.507621,...,-0.501242,-0.535019,0.553492,-0.096223,-0.314923,-0.684897,-1.044802,0.414803,0.824820,0.0


In [17]:
import pandas as pd

csv_filename='/blue/xiaofan/sunhuaikuan/PAPERS/DNA_LLM_REVIEW/embeddings/pathogenecity/embedding-csv/pathogenecity_gpn_noncoding-0.csv'

def load_embedding_file(csv_filename):

    df=pd.read_csv(csv_filename)
    
    # new_arr = data_array[:, :-4]
    # last_column = data_array[:, -1]
    # data_array = np.column_stack((new_arr, last_column))
    
    column_names = [f'{i}' for i in range(1, df.shape[1])]
    column_names.extend([ 'y'])
    
    df.columns = column_names
    return df

df = load_embedding_file(csv_filename)
df

,1,2,3,4,5,6,7,8,9,10,...,762,763,764,765,766,767,768,769,770,y
0,0.445931,0.805443,0.499554,0.711929,0.321147,-1.092941,-0.199364,0.184887,0.943818,0.292585,...,-0.415482,-0.197059,-0.079640,-1.064177,-0.466612,-0.472388,0.848660,1,128,0
1,0.461076,0.809369,0.483196,0.685488,0.303076,-1.104358,-0.194929,0.190177,0.957248,0.253928,...,-0.387591,-0.193366,-0.085549,-1.059689,-0.472710,-0.470236,0.855772,1,128,0
2,0.513312,0.754226,0.497715,0.698823,0.286727,-1.017692,-0.148586,0.149295,0.969672,0.267036,...,-0.378097,-0.163968,-0.135677,-1.037855,-0.503077,-0.470197,0.829473,1,128,0
3,0.513312,0.754226,0.497715,0.698823,0.286727,-1.017692,-0.148586,0.149295,0.969672,0.267036,...,-0.378097,-0.163968,-0.135677,-1.037855,-0.503077,-0.470197,0.829473,1,128,0
4,0.366928,0.121256,0.285477,1.180893,-0.379903,-0.543935,-0.505797,0.111189,1.207105,0.233208,...,-0.042508,-0.255080,-0.285948,-0.815723,-0.524366,-0.180991,0.821893,1,128,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95754,0.960115,-0.405934,-0.963120,0.604153,-1.069464,-0.038014,0.032151,0.178382,-0.173240,-0.667455,...,1.289428,0.359950,-0.283300,-0.062038,-1.202238,0.390647,0.621940,X,128,0
95755,0.015163,-0.034635,-0.535783,0.233521,-0.565940,-0.448956,-0.256404,0.180318,-0.123829,-0.844805,...,0.556938,-0.078505,-0.648336,-0.629008,-1.000944,0.622980,0.749247,X,128,0
95756,-0.002326,0.007319,-0.560664,-0.010452,-0.207512,-0.635047,-0.444733,0.240929,-0.179017,-0.806953,...,0.526378,0.160796,-0.679473,-0.535794,-0.854475,0.533914,0.525361,X,128,0
95757,0.143588,0.178700,-0.840755,0.159178,-0.592489,-0.772373,-0.460265,0.607133,-0.207599,-0.507621,...,0.553492,-0.096223,-0.314923,-0.684897,-1.044802,0.414803,0.824820,X,128,0
